<a href="https://colab.research.google.com/github/nicologhielmetti/AN2DL-challenges/blob/master/challenge3/chall3.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [2]:
import json, os
from functools import partial
import numpy as np
import tensorflow as tf
from PIL import Image
import random
import keras.layers as layers
import keras.models as models
from keras.initializers import orthogonal
from keras.optimizers import Adam
import shutil

In [3]:
print(tf.__version__)

2.4.1


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1tglwr5cbQbzrSLmJlHmz33htFUw0yzc4
!unzip -qq /content/anndl-2020-vqa.zip

Downloading...
From: https://drive.google.com/uc?id=1tglwr5cbQbzrSLmJlHmz33htFUw0yzc4
To: /content/anndl-2020-vqa.zip
4.26GB [01:16, 56.0MB/s]


In [6]:
#!unzip -qq VQA_Dataset.zip -d VQA_Dataset

In [7]:
def create_split_files(file_path, train_val_split):
  with open(file_path,'r') as json_dataset:
    data = json.load(json_dataset)

    tot_list = []
    for k,v in data.items():
      tot_list.append({k:v})
    
    train_list = random.sample(tot_list, int(len(tot_list) * (1 - train_val_split)))
    validation_list = [x for x in tot_list if x not in train_list]

    with open("VQA_Dataset/train.json", "w") as train:
        json.dump(train_list, train)
        train.close()
    with open("VQA_Dataset/valid.json", "w") as validation:
        json.dump(validation_list, validation)
        validation.close()
    
    #assert len([x for x in tot_list if x in train_list and x in validation_list]) == 0

def create_train_test_dirs(json_definition, dataset_path, split_name):
    dest_dir = os.path.join(dataset_path, split_name)
    if not os.path.isdir(dest_dir):
      os.mkdir(dest_dir)
      os.mkdir(os.path.join(dest_dir, split_name))
    if split_name == 'test':
      for k,v in json_definition.items():
        try:
            shutil.move(
                os.path.join(dataset_path, "Images", v['image_id']+'.png'),
                os.path.join(dest_dir, split_name, v['image_id']+'.png')
            )
        except FileNotFoundError as e:
            print("Split name: " + split_name + ". File not found: " + str(e))
            continue
        
    else:
      for elem in json_definition:
          #print(os.path.join(dataset_path, "Images", elem[list(elem.keys())[0]]['image_id'][:-1]+'.png'))
          try:
              shutil.move(
                  os.path.join(dataset_path, "Images", elem[list(elem.keys())[0]]['image_id']+'.png'),
                  os.path.join(dest_dir, split_name, elem[list(elem.keys())[0]]['image_id']+'.png')
              )
          except FileNotFoundError as e:
              print("Split name: " + split_name + ". File not found: " + str(e))
              continue


In [ ]:
img_size = (256, 256)

random.seed(96)

cwd = os.getcwd()
datasetName = os.path.join(cwd,'VQA_Dataset')
trainJsonName = 'train.json'
validJsonName = 'valid.json'
testJsonName  = 'test_questions.json'
#os.chdir(datasetName)
imagesPath = os.path.join(datasetName, 'Images')
trainJsonPath = os.path.join(datasetName, trainJsonName)
validJsonPath = os.path.join(datasetName, validJsonName)
testJsonPath  = os.path.join(datasetName, testJsonName)


create_split_files(os.path.join(datasetName, 'train_questions_annotations.json'), 0.5)

with open(trainJsonPath,'r') as json_file_train, open(validJsonPath, 'r') as json_file_valid, open(testJsonPath, 'r') as json_file_test:
    data_train = json.load(json_file_train)
    data_valid = json.load(json_file_valid)
    data_test = json.load(json_file_test)

    json_file_train.close()
    json_file_valid.close()
    json_file_test.close()
    
os.chdir(cwd)
create_train_test_dirs(data_train, datasetName, 'train')
create_train_test_dirs(data_valid, datasetName, 'validation')
create_train_test_dirs(data_test, datasetName, 'test')


In [9]:
preproc_fun_fixed = partial(tf.keras.preprocessing.image.smart_resize, size=img_size)
batch_size = 32
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, data_format='channels_last', preprocessing_function=preproc_fun_fixed)
train_data = datagen.flow_from_directory(datasetName+'/train', img_size, class_mode='input', batch_size=batch_size)
valid_data = datagen.flow_from_directory(datasetName+'/validation', img_size, class_mode='input', batch_size=batch_size)

Found 20833 images belonging to 1 classes.
Found 7911 images belonging to 1 classes.


In [10]:
def Conv2DLayer(x, filters, kernel, strides, padding, block_id, kernel_init=orthogonal()):
    prefix = f'block_{block_id}_'
    x = layers.Conv2D(filters, kernel_size=kernel, strides=strides, padding=padding,
                      kernel_initializer=kernel_init, name=prefix+'conv')(x)
    x = layers.LeakyReLU(name=prefix+'lrelu')(x)
    x = layers.Dropout(0.2, name=prefix+'drop')((x))
    x = layers.BatchNormalization(name=prefix+'conv_bn')(x)
    return x

def Transpose_Conv2D(x, filters, kernel, strides, padding, block_id, kernel_init=orthogonal()):
    prefix = f'block_{block_id}_'
    x = layers.Conv2DTranspose(filters, kernel_size=kernel, strides=strides, padding=padding,
                               kernel_initializer=kernel_init, name=prefix+'de-conv')(x)
    x = layers.LeakyReLU(name=prefix+'lrelu')(x)
    x = layers.Dropout(0.2, name=prefix+'drop')((x))
    x = layers.BatchNormalization(name=prefix+'conv_bn')(x)
    return x



def AutoEncoder(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    # 256 x 256
    conv1 = Conv2DLayer(inputs, 64, 3, strides=1, padding='same', block_id=1)
    conv2 = Conv2DLayer(conv1, 64, 3, strides=2, padding='same', block_id=2)
    
    # 128 x 128
    conv3 = Conv2DLayer(conv2, 128, 5, strides=2, padding='same', block_id=3)
    
    # 64 x 64
    conv4 = Conv2DLayer(conv3, 128, 3, strides=1, padding='same', block_id=4)
    conv5 = Conv2DLayer(conv4, 256, 5, strides=2, padding='same', block_id=5)
    
    # 32 x 32
    conv6 = Conv2DLayer(conv5, 512, 3, strides=2, padding='same', block_id=6)
    
    # 16 x 16
    deconv1 = Transpose_Conv2D(conv6, 512, 3, strides=2, padding='same', block_id=7)
    
    # 32 x 32
    skip1 = layers.concatenate([deconv1, conv5], name='skip1')
    conv7 = Conv2DLayer(skip1, 256, 3, strides=1, padding='same', block_id=8)
    deconv2 = Transpose_Conv2D(conv7, 128, 3, strides=2, padding='same', block_id=9)
    
    # 64 x 64
    skip2 = layers.concatenate([deconv2, conv3], name='skip2')
    conv8 = Conv2DLayer(skip2, 128, 5, strides=1, padding='same', block_id=10)
    deconv3 = Transpose_Conv2D(conv8, 64, 3, strides=2, padding='same', block_id=11)
    
    # 128 x 128
    skip3 = layers.concatenate([deconv3, conv2], name='skip3')
    conv9 = Conv2DLayer(skip3, 64, 5, strides=1, padding='same', block_id=12)
    deconv4 = Transpose_Conv2D(conv9, 64, 3, strides=2, padding='same', block_id=13)
    
    # 256 x 256
    skip3 = layers.concatenate([deconv4, conv1])
    conv10 = layers.Conv2D(3, 3, strides=1, padding='same', activation='sigmoid',
                       kernel_initializer=orthogonal(), name='final_conv')(skip3)

    
    return models.Model(inputs=inputs, outputs=conv10)

In [11]:
autoencoder = AutoEncoder((*img_size, 3))
model_opt = Adam(lr=0.002)

autoencoder.compile(optimizer=model_opt, loss='mse', metrics=['accuracy'])
#autoencoder.summary()

In [12]:
saved_weight = os.path.join('/content/drive/MyDrive','saved_models_chall3', 'dataweights.{epoch:02d}.hdf5')
modelchk = tf.keras.callbacks.ModelCheckpoint(saved_weight, 
                                              monitor='val_loss', 
                                              verbose=1,
                                              save_best_only=True, 
                                              save_weights_only=False
                                              )

tensorboard = tf.keras.callbacks.TensorBoard(log_dir='logs',
                                          histogram_freq=0,
                                          write_graph=True,
                                          write_images=True
                                          )

csv_logger = tf.keras.callbacks.CSVLogger('logs/keras_log.csv',
                                       append=True)

es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)


In [13]:
autoencoder = tf.keras.models.load_model(os.path.join('/content/drive/MyDrive','saved_models_chall3', 'dataweights.02.hdf5'))

In [14]:
epochs=50
autoencoder.fit(train_data,
                steps_per_epoch = len(train_data),
                epochs=epochs,
                verbose=1,
                validation_data=valid_data,
                validation_steps = len(train_data),
                callbacks=[modelchk,tensorboard, csv_logger, es_callback]
                )

Epoch 1/50
652/652 [==============================] - 700s 1s/step - loss: 7.3180e-04 - accuracy: 0.9569 - val_loss: 5.4506e-04 - val_accuracy: 0.9613

Epoch 00001: val_loss improved from inf to 0.00055, saving model to /content/drive/MyDrive/saved_models_chall3/dataweights.01.hdf5
Epoch 2/50
652/652 [==============================] - 560s 859ms/step - loss: 6.9260e-04 - accuracy: 0.9582


KeyError: ignored